# Ipython of Off-line Minituar leg jumping with stairs analysis#
## Nov. 30, 2017##
This file is the data analysis of real-time data captured by Qualisys Camera, in 1kHz. And Arduino board's signal is streamed at the same time. 

In [1]:
%matplotlib 

Using matplotlib backend: Qt5Agg


 to view the result, please read
    - trail one:
        20171130-184552_deadbeat_v0.1.2_Qua_collect1.csv;
        20171130-184552_deadbeat_v0.1.2_Ard_collect1.csv;
    - trail two:
        20171130-185826_deadbeat_v0.1.2_Qua_collect2.csv;
        20171130-185826_deadbeat_v0.1.2_Ard_collect2.csv;
    - trail three:
        20171130-192820_deadbeat_v0.1.2_Qua_collect3.csv;
        20171130-192820_deadbeat_v0.1.2_Ard_collect3.csv;

In [39]:
import pandas as pd
import numpy as np
import read as rd
import matplotlib.pyplot as plt
from numpy import linalg as LA

df_q = pd.read_csv('20171207-114005_deadbeat_v0.1.2_Qua_collect2.csv') # jump one time per stair change
df_a = pd.read_csv('20171207-114005_deadbeat_v0.1.2_Ard_collect2.csv')

## Qualisys

In [40]:
df_q.head()

,Unnamed: 0,foot_x,foot_y,led_x,top_x,top_y
0,0,-271.919830,-235.729919,NaN,-978.343506,-215.714874
1,1,-271.885315,-235.695435,NaN,-978.308655,-215.749435
2,2,-271.885315,-235.695435,NaN,-978.343445,-215.680252
3,3,-271.919830,-235.729919,NaN,-978.343628,-215.749496
4,4,-271.989044,-236.109131,NaN,-978.308655,-215.749435


In [41]:
led = df_q.led_x

LED is the signal of an LED, when arduino start recording, LED truns on so that Qualysis camera captured the light. This is for synchronizing 
time of Arduino and Qualysis data.

In [42]:
LED = np.array(led[0:-1]);
tmp = np.argwhere(np.isfinite(LED))
b = tmp[0][0] # b is the start time for camera.

In [43]:
top_x = df_q.top_x;
top_y = df_q.top_y;
foot_x = df_q.foot_x;
foot_y = df_q.foot_y;

TOP_X = np.array(top_y[b:-1]);
TOP_Y = np.array(top_x[b:-1]);
FOOT_X = np.array(foot_y[b:-1]);
FOOT_Y = np.array(foot_x[b:-1]);

In [44]:

p1, p2 = np.zeros((len(TOP_X),2)), np.zeros((len(TOP_X),2))
for i in range(len(TOP_X)):
    p1[i]=[TOP_X[i],TOP_Y[i]];
    p2[i]=[FOOT_X[i],FOOT_Y[i]];
# p1, p2 are top and foot markers, TIME_Q is the time for Qualysis
TIME_Q= np.array((tmp-tmp[0])*0.001)
#Analyse if Time and point has same length
np.isclose(len(TIME_Q),len(TOP_X))

True

## Callibration points in Qualysis

In [45]:
P1 = np.array([ -281.27318135, -267.38286831]);
P2 = np.array([  25.98821099, -565.81275512]);
P3 = np.array([  19.61096392, -272.63622981]); # P3 is the origin of coordinator

top_cal and foot_cal are calibrated (x,y) coordinates of top and foot marker

In [46]:
top_cal = rd.calibration(p1, P1, P2, P3)
foot_cal = rd.calibration(p2, P1, P2, P3)

In [47]:
plt.figure(figsize=(14,14));
plt.title('trial two Minituar jumping real-time capture');
plt.plot(TIME_Q, top_cal[:,1],label="top")
plt.plot(TIME_Q, foot_cal[:,1],label="foot")
plt.xlabel('time/s');
plt.ylabel('height/mm');

plt.legend(loc = 'upper right')
plt.show()

## Arduino

In [48]:
df_a.head()

,Unnamed: 0,ard,time
0,0,"1.3803,1.8976,0\r\n",0.997
1,1,"1.5071,2.0064,0\r\n",1.007
2,2,"1.6161,2.1186,0\r\n",1.017
3,3,"1.7155,2.2289,0\r\n",1.027
4,4,"1.8154,2.3333,0\r\n",1.037


In [49]:
Ard = df_a.ard;
time_ard = df_a.time

In [50]:
ARD = np.array(Ard);
TIME_ARD = np.array(time_ard)

In [51]:
def jump_from_csv(ARD, height):
    # This function is to read csv data from arduino csvs.
    # Output
    #     theta_1: Angle of Motor1
    #     theta_2: Angle of Motor2
    #     stair: testbed motor command
    #     real_height: actual height of terrain
            
    length = len(ARD);
    theta_1, theta_2, stair, real_height = np.zeros(length), np.zeros(length), np.zeros(length), np.zeros(length);
    for i in range(length):
        ard_line = ARD[i];
        if (len(ard_line.split(",")) == 3):
            try:
                theta_1[i] = float(ard_line.split(",")[0])
                theta_2[i] = float(ard_line.split(",")[1])
                stair[i] = int(ard_line.split(",")[2])
                #print stair[i]
                real_height[i] = height[int(stair[i])];
            except ValueError:
                print i,'ValueError'
        else:
            print i,'less than 3 value'
               
    return theta_1, theta_2, stair, real_height

height = [28, 18, -2, 8,-2]
theta_1, theta_2, stair, real_height= jump_from_csv(ARD, height)


## combine and cut

set the stair height index

In [52]:
plt.figure(figsize=(14,14));
plt.title('trial three Minituar jumping real-time capture');
plt.plot(TIME_Q, top_cal[:,1],label="top")
plt.plot(TIME_Q, foot_cal[:,1],label="foot")
plt.plot(TIME_ARD, real_height,label="stair")
plt.xlabel('time/s');
plt.ylabel('height/mm');

plt.legend(loc = 'upper right')
plt.show()

find changes in commands

In [53]:
def pick_stairs (stair, TIME_ARD):
    # This function is to find terrain transition time
    #     Output:
    #         tran_num: Number of transition
    #         index_change: the index in Arduino list when transition happens
    #         time_change: the time of change according to index_change
    
    tran_num = 1; # times of change
    index_change = [0]
    time_change = [0]
    for i in range(len(stair)-1):
        if not np.isclose(stair[i], stair[i+1]):
            tran_num += 1;
            index_change.append(i)
            time_change.append(TIME_ARD[i])
            #print tran_num
    return tran_num, index_change, time_change


tran_num, index_change, time_change = pick_stairs (stair, TIME_ARD)


In [54]:
index_change[0:-1]

[0,
 123,
 673,
 1223,
 1773,
 2323,
 2873,
 3423,
 3973,
 4523,
 5073,
 5623,
 6173,
 6723,
 7273,
 7823,
 8373,
 8923,
 9473,
 10023,
 10573,
 11123]

In [55]:
top_cal.shape

(123918L, 2L)

In [56]:
def cut_data(TIME_ARD, top_cal, foot_cal, index_change, time_change, tran_num, theta_1, theta_2):
#     This func is to devide Arduino and Qualysis data into different terrain transition
#     Output:
#         CUT_FOOT[i,[x,y]]: Matrix of Foot (x,y) in total i transitions
#         CUT_TOP[i,[x,y]]: Matrix of Top (x,y) in total i transitions
#         Time_Q[i,t]: Matrix of time t(0, 15000) in total i transitions
#         CUT_Ard[i,[theta_1,theta_2]]: Matrix of (theta_1, theta_2) in total i transitions
#         Time_A[i,t]: Matrix of time t(0,1500) in total i transitions
    
    CUT_FOOT = np.zeros((tran_num, 15000, 2)); # height of foot
    #CUT_FOOT_Y = np.zeros((tran_num, 15000, 2));
    CUT_TOP = np.zeros((tran_num, 15000, 2)); # top of foot
    #CUT_TOP_Y = np.zeros((tran_num, 15000, 2)); # top of foot
    Time_Q = np.zeros((tran_num, 15000));
    CUT_Ard = np.zeros((tran_num, 1500, 2))
    Time_A = np.zeros((tran_num, 1500));
    #bacause we regard qualisys time is strictly 0.001 sec difference
    a = 0;
    for i in range(len(index_change)-1):
        
        for j in range(int(float('%.3f'% time_change[i])*1000), int(float('%.3f'% time_change[i+1])*1000)):
            k = j-int(float('%.3f'% time_change[i])*1000);
            CUT_FOOT[a,k,0] = foot_cal[j,0];
            CUT_FOOT[a,k,1] = foot_cal[j,1];
            CUT_TOP[a,k,0] = top_cal[j,0];
            CUT_TOP[a,k,1] = top_cal[j,1];
            Time_Q[a,k] = k*0.001
        a = a + 1;
    for i in range(len(index_change)-1):
        for j in range(index_change[i], index_change[i+1]):
            k = j - index_change[i];
            CUT_Ard[i,k,0] = theta_1[j];
            CUT_Ard[i,k,1] = theta_2[j];
            Time_A[i,k] = TIME_ARD[j] - TIME_ARD[index_change[i]]
    
    return CUT_FOOT, CUT_TOP, Time_Q, CUT_Ard, Time_A

CUT_FOOT, CUT_TOP, Time_Q, CUT_Ard, Time_A = cut_data(TIME_ARD, top_cal, foot_cal, index_change, time_change, tran_num, theta_1, theta_2)

In [57]:
real_height

array([ 28.,  28.,  28., ...,  28.,  28.,  28.])

In [58]:
i =3;
def nozero(x):
    index = np.where(x == 0)[0];
    
    return index[0]-1

plt.figure(figsize=(14,14));
plt.title('trial one Minituar jumping real-time capture %d th transition, stair height = %f mm, Qualysis, 1k Hz'% (i,real_height[index_change[i+1]]));
zero = nozero(CUT_FOOT[i,:,1])
# time_q = nozero(Time_Q[i,:])
# top = nozero(CUT_TOP[i,:,1])

# plt.plot(time_q ,foot, 'g' ,label="foot")
# plt.plot(time_q ,top, 'r',label="top")

plt.plot(Time_Q[i,0:zero] ,CUT_FOOT[i,0:zero,1], 'g' ,label="foot")
plt.plot(Time_Q[i,0:zero] ,CUT_TOP[i,0:zero,1], 'r',label="top")
plt.xlabel('time/s');
plt.ylabel('height/mm');

plt.legend(loc = 'upper right')

plt.show()

In [59]:
plt.figure(figsize=(14,14));
plt.title('trial one Minituar jumping real-time capture %d th transition, stair height = %f mm, Arduino, 100Hz'% (i,real_height[index_change[i+1]]));
zero_ard = nozero(CUT_Ard[i,:,0])-1
plt.plot(Time_A[i,0:zero_ard] ,CUT_Ard[i,0:zero_ard,0], 'b', label="Arduino_angle_1")
plt.plot(Time_A[i,0:zero_ard] ,CUT_Ard[i,0:zero_ard,1], 'r', label="Arduino_angle_2")
plt.xlabel('time/s');
plt.ylabel('angle/rad');

plt.legend(loc = 'upper right')

plt.show()

Save as json

In [60]:
import json

In [61]:
Raw = {'Qualysis':{'Time_Q': TIME_Q.tolist(),'TOP': top_cal.tolist(),'FOOT': foot_cal.tolist(),
                  'Note': 'Time_Q: time scale of Qualysis, TOP: (x,y) coordinates of top marker, FOOT: (x,y) coordinates of buttom marker'},
       'Arduino':{'Time_A': TIME_ARD.tolist(),'theta_1': theta_1.tolist(),'theta_2': theta_2.tolist(),'stair': stair.tolist(),
                 'Note': 'Time_ARD: time scale of Arduino, theta_1: angle of Motor1, theta_2: angle of Motor2, stair: testbed motor command'}};
Cut = {'Qualysis':{'CUT_FOOT': CUT_FOOT.tolist(),'CUT_TOP': CUT_TOP.tolist(),'Time_Q': Time_Q.tolist(),
                  'Note': 'CUT_FOOT[i,[x,y]]: Matrix of Foot (x,y) in total i transitions, CUT_TOP[i,[x,y]]: Matrix of Top (x,y) in total i transitions, Time_Q[i,t]: Matrix of time t(0, 15000) in total i transitions '},
       'Arduino':{'Cut_theta': CUT_Ard.tolist(),'Time_A': Time_A.tolist(),'tran_index': index_change,'stair': real_height.tolist(),
                 'Note': 'CUT_Ard[i,[theta_1,theta_2]]: Matrix of (theta_1, theta_2) in total i transitions, Time_A[i,t]: Matrix of time t(0,1500) in total i transitions, tran_index: the index in Arduino list when transition happens, stair: actual height of terrain'}};

In [62]:
import pickle
djson = {'Raw_data':Raw,
         'Cut_data':Cut};
pickle.dump( djson, open( "trail2_jump10.json", "wb" ) )